## [ 1. Python으로 MongoDB 제어하기 ]

### (1) pymongo 라이브러리 설치
> pip install pymongo

### (2) Mongodb 접속
-----
>import pymongo
>객체변수 = pymongo.MongoClient(mongo_server. 27017)

- 따로, 서버 ip를 지정하지 않을 경우 사용하는 컴퓨터의 로컬ip가 들어간다.
    - ex) connection = pymongo.MongoClient()



- AWS EC2 서버에서 접속을 원할 경우, 'mongodb://AWS_EC2_IP' 를 ()에 넣으면된다.
    - ex) connection = pymongo.MongoClient('mongodb://15.164.160.186')

In [4]:
import pymongo
connection = pymongo.MongoClient()


### (3) Database 생성/연결
-----
- conn_db 변수에 test 데이터베이스의 객체가 들어간다.
- test라는 데이터베이스가 해당 ip에 없을경우 생성된다.

In [5]:
conn_db = connection.test

In [ ]:
conn_db = connection['test'] # 이렇게도 선언 가능하다.

-----
- print를 사용하여 database가 저장된 객체를 출력하면, 출력된 MongoClient함수를 통해서 해당 database가 저장된 host가 어딘지 확인 할 수 있다.

In [7]:
print(conn_db)

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'test')


In [8]:
print(dir(conn_db)) 

['_BaseObject__codec_options', '_BaseObject__read_concern', '_BaseObject__read_preference', '_BaseObject__write_concern', '_Database__client', '_Database__incoming_copying_manipulators', '_Database__incoming_manipulators', '_Database__name', '_Database__outgoing_copying_manipulators', '_Database__outgoing_manipulators', '__call__', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattr__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_apply_incoming_copying_manipulators', '_apply_incoming_manipulators', '_command', '_create_or_update_user', '_current_op', '_default_role', '_fix_incoming', '_fix_outgoing', '_list_collections', '_read_preference_for', '_retryable_read_command', '_write_concern_for'

----
- name을 사용하여 해당 데이터베이스의 이름이 확인 가능하다.

In [9]:
print(conn_db.name)

test
